In [1]:
from keras.applications import vgg16
from keras.preprocessing.image import img_to_array,ImageDataGenerator
from keras.models import Sequential,Model
from keras.layers import Input,Dense,Conv2D,MaxPooling2D,Flatten
from keras.optimizers import adam

import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [2]:
vgg16=vgg16.VGG16(weights='imagenet')

553467904/553467096 [==============================] - 528s 1us/step


In [3]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [39]:
img_rows, img_cols = 224, 224 
train_dir='C:///Users/Acer/Desktop/mlops/face/train'
validation_dir='C:///Users/Acer/Desktop/mlops/face/validate'
num_train_img= 256
num_valid_img= 3
epoch= 5

In [40]:
x=vgg16.get_layer('fc2').output

In [41]:
out1=Dense(3,activation='softmax',name='out_layer')(x)

In [42]:
final_model=Model(input=vgg16.input,outputs=out1)

C:\Users\Acer\anaconda3\envs\MLOps\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ou..., inputs=Tensor("in...)`
  """Entry point for launching an IPython kernel.


In [43]:
final_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [44]:
final_model.layers[0].name

'input_1'

In [45]:
for layer in final_model.layers:
    if layer.name not in ['out_layer']:
        layer.trainable=False

In [46]:
for layer in final_model.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True


In [47]:
final_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [48]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory(
        train_dir,
        target_size=[img_rows,img_cols],
        class_mode='categorical')
test_set = test_datagen.flow_from_directory(
        validation_dir,
        target_size=[img_rows,img_cols],
        class_mode='categorical')

Found 242 images belonging to 3 classes.
Found 24 images belonging to 3 classes.


In [49]:
training_set.class_indices

{'0': 0, '1': 1, '2': 2}

In [50]:
from keras.optimizers import Adam

In [51]:
final_model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [52]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("FaceRegModel.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

callbacks = [earlystop, checkpoint]

In [54]:
final_model.fit(training_set,epochs=epoch,steps_per_epoch=80)

Epoch 1/5
80/80 [==============================] - 1033s 13s/step - loss: 1.0967 - accuracy: 0.3773
Epoch 2/5
80/80 [==============================] - 1039s 13s/step - loss: 1.0905 - accuracy: 0.3860
Epoch 3/5
80/80 [==============================] - 1202s 15s/step - loss: 1.0830 - accuracy: 0.4066
Epoch 4/5
80/80 [==============================] - 1248s 16s/step - loss: 1.0776 - accuracy: 0.4079
Epoch 5/5
80/80 [==============================] - 1153s 14s/step - loss: 1.0675 - accuracy: 0.4479


In [58]:
final_model.save('myFacialExpressionModel.h5')

In [59]:
from keras.preprocessing import image

In [128]:
test_image = image.load_img('C:///Users/Acer/Desktop/WIN_202018.jpg', 
               target_size=(224,224))

In [129]:
#plt.imshow(test_image)

In [130]:
test_image = image.img_to_array(test_image)

In [131]:
test_image = np.expand_dims(test_image, axis=0)

In [136]:
test_image.shape

(1, 224, 224, 3)

In [137]:
def printClass(res):
  if res==0:
    print('Yashi')
  elif res==1:
    print('Shruti')
  elif res==2:
    print('Subhadeep')

In [138]:
res = np.argmax(final_model.predict(test_image))

In [139]:
printClass(res)

Shruti
